In [89]:
import os
import pickle
import numpy as np
from PIL import Image
import multiprocessing as mp

import torch
import torchvision
from torch.utils.data.dataset import Dataset

from pyntcloud import PyntCloud

import glob
import math

In [90]:
def get_batches(data_pcl_torch, batch_size = 20, max_point_num = 150000, grid_size=1000):
    
    data_num = np.zeros((batch_size), dtype=np.int32)
    indices_split_to_full = np.zeros((batch_size, max_point_num), dtype=np.int32)
    
    data_pcl = data_pcl_torch.numpy()
    print(data_pcl.shape)
    
    data_pcl_torch = data_pcl_torch.unsqueeze(0)
    data_pcl_torch = data_pcl_torch.numpy()
    
    xyz =data_pcl[:,0:3]
    i = data_pcl[:,3]
    xyz_min = np.amin(xyz, axis=0, keepdims=True)
    print("min ", xyz_min)
    xyz_max = np.amax(xyz, axis=0, keepdims=True)
    print("max ", xyz_max)
    block_size = (2 * (xyz_max[0, 0] - xyz_min[0, 0]), 2 * (xyz_max[0, 1] - xyz_min[0, 1]) ,  2 * (xyz_max[0, -1] - xyz_min[0, -1]))
    
    xyz_blocks = np.floor((xyz - xyz_min) / block_size).astype(np.int)
    print("block size ", block_size)
    print("diff block ",(xyz - xyz_min))
    #print('{}-Collecting points belong to each block...'.format(datetime.now(), xyzrcof.shape[0]))
    print(" blocks ",xyz_blocks)
    print(np.unique(xyz_blocks, return_inverse=True,return_counts=True, axis=0))
    blocks, point_block_indices, block_point_counts = np.unique(xyz_blocks, return_inverse=True,
                                                                return_counts=True, axis=0)
    block_point_indices = np.split(np.argsort(point_block_indices), np.cumsum(block_point_counts[:-1]))
    #print('{}-{} is split into {} blocks.'.format(datetime.now(), dataset, blocks.shape[0]))

    block_to_block_idx_map = dict()
    for block_idx in range(blocks.shape[0]):
        block = (blocks[block_idx][0], blocks[block_idx][1])
        block_to_block_idx_map[(block[0], block[1])] = block_idx

    # merge small blocks into one of their big neighbors
    block_point_count_threshold = max_point_num / 3
    print("threshold ", block_point_count_threshold)
    #print("block_point_count_threshold",block_point_count_threshold)
    nbr_block_offsets = [(0, 1), (1, 0), (0, -1), (-1, 0), (-1, 1), (1, 1), (1, -1), (-1, -1)]
    block_merge_count = 0
    print("blocking")
    print(block_point_counts)
    print(point_block_indices)
    print(blocks)
    for block_idx in range(blocks.shape[0]):
        if block_point_counts[block_idx] >= block_point_count_threshold:
            print("here")
            print(block_idx, block_point_counts[block_idx])

            continue


        block = (blocks[block_idx][0], blocks[block_idx][1])
        for x, y in nbr_block_offsets:
            nbr_block = (block[0] + x, block[1] + y)
            if nbr_block not in block_to_block_idx_map:
                continue

            nbr_block_idx = block_to_block_idx_map[nbr_block]
            if block_point_counts[nbr_block_idx] < block_point_count_threshold:
                continue


            #print(block_idx, nbr_block_idx, block_point_counts[nbr_block_idx])

            block_point_indices[nbr_block_idx] = np.concatenate(
                [block_point_indices[nbr_block_idx], block_point_indices[block_idx]], axis=-1)
            block_point_indices[block_idx] = np.array([], dtype=np.int)
            block_merge_count = block_merge_count + 1
            break
    #print('{}-{} of {} blocks are merged.'.format(datetime.now(), block_merge_count, blocks.shape[0]))

    idx_last_non_empty_block = 0
    for block_idx in reversed(range(blocks.shape[0])):
        if block_point_indices[block_idx].shape[0] != 0:
            idx_last_non_empty_block = block_idx
            break

    # uniformly sample each block
    for block_idx in range(idx_last_non_empty_block + 1):
        point_indices = block_point_indices[block_idx]
        if point_indices.shape[0] == 0:
            continue

        #print(block_idx, point_indices.shape)
        block_points = xyz[point_indices]
        block_min = np.amin(block_points, axis=0, keepdims=True)
        xyz_grids = np.floor((block_points - block_min) / grid_size).astype(np.int)
        grids, point_grid_indices, grid_point_counts = np.unique(xyz_grids, return_inverse=True,
                                                                 return_counts=True, axis=0)
        grid_point_indices = np.split(np.argsort(point_grid_indices), np.cumsum(grid_point_counts[:-1]))
        grid_point_count_avg = int(np.average(grid_point_counts))
        point_indices_repeated = []
        for grid_idx in range(grids.shape[0]):
            point_indices_in_block = grid_point_indices[grid_idx]
            repeat_num = math.ceil(grid_point_count_avg / point_indices_in_block.shape[0])
            if repeat_num > 1:
                point_indices_in_block = np.repeat(point_indices_in_block, repeat_num)
                np.random.shuffle(point_indices_in_block)
                point_indices_in_block = point_indices_in_block[:grid_point_count_avg]
            point_indices_repeated.extend(list(point_indices[point_indices_in_block]))
        block_point_indices[block_idx] = np.array(point_indices_repeated)
        block_point_counts[block_idx] = len(point_indices_repeated)

    idx = 0
    for block_idx in range(idx_last_non_empty_block + 1):
        point_indices = block_point_indices[block_idx]
        if point_indices.shape[0] == 0:
            continue

        block_point_num = point_indices.shape[0]
        block_split_num = int(math.ceil(block_point_num * 1.0 / max_point_num))
        point_num_avg = int(math.ceil(block_point_num * 1.0 / block_split_num))
        point_nums = [point_num_avg] * block_split_num
        point_nums[-1] = block_point_num - (point_num_avg * (block_split_num - 1))
        starts = [0] + list(np.cumsum(point_nums))

        np.random.shuffle(point_indices)
        block_points = xyz[point_indices]


        block_min = np.amin(block_points, axis=0, keepdims=True)
        block_max = np.amax(block_points, axis=0, keepdims=True)
        #block_center = (block_min + block_max) / 2
        #block_center[0][-1] = block_min[0][-1]
        #block_points = block_points - block_center  # align to block bottom center
        x, y, z = np.split(block_points, (1, 2), axis=-1)
        print(i.shape)
        print(x.shape)
        block_xzyrgbi = np.concatenate([x, z, y, i[:, np.newaxis][point_indices]], axis=-1)
        print(block_xzyrgbi.shape)
        for block_split_idx in range(block_split_num):
            start = starts[block_split_idx]
            point_num = point_nums[block_split_idx]
            #print(block_split_num, block_split_idx, point_num )



            end = start + point_num
            idx_in_batch = idx % batch_size
            print(idx_in_batch)
            data_pcl_torch[idx_in_batch, 0:point_num, ...] = block_xzyrgbi[start:end, :]
            data_num[idx_in_batch] = point_num
            indices_split_to_full[idx_in_batch, 0:point_num] = point_indices[start:end]

            #print("indices_split_to_full", idx_in_batch, point_num, indices_split_to_full)

            if  (block_idx == idx_last_non_empty_block and block_split_idx == block_split_num - 1): #Last iteration

                item_num = idx_in_batch + 1
                
            idx = idx + 1
    return data_pcl_torch, data_num, indices_split_to_full, item_num


In [85]:
load_file = np.fromfile(os.path.join("000000.bin"),dtype=np.float32).reshape((-1, 4))

In [91]:
main_tensor = np.arange(150000*4).reshape((150000,4))
# main_tensor = load_file[:, np.newaxis]
main_tensor = load_file
batch_size = 20
# main_tensor[:full_data.shape[0], :full_data.shape[1]] = full_data
data_pcl = torchvision.transforms.ToTensor()(main_tensor)
print("pcl shape ",data_pcl.shape)
for each_batch in data_pcl:
    batched_data, data_num, indices_split_to_full, item_num = get_batches(each_batch, batch_size = batch_size)
    print(batched_data.shape)
    print(data_num.shape)
    print(indices_split_to_full.shape)
    print(item_num)
    print(batched_data)
    
    data =batched_data[0:item_num, ...].astype(np.float32) 
    print(data.shape)

    data_num =data_num[0:item_num, ...] 
    print(data_num.shape)
    indices_split_to_full = indices_split_to_full[0:item_num]
    print(indices_split_to_full.shape)

    batch_num = data.shape[0]
    for batch_idx in range(batch_num):
        points_batch = data[[batch_idx] * batch_size, ...]
        print(points_batch.shape)
        point_num = data_num[batch_idx]
        print(point_num)
        break

        tile_num = int ( math.ceil((sample_num * batch_size) / point_num) )
        indices_shuffle = np.tile(np.arange(point_num), tile_num)[0:sample_num * batch_size]
        np.random.shuffle(indices_shuffle)
        indices_batch_shuffle = np.reshape(indices_shuffle, (batch_size, sample_num, 1))
        indices_batch = np.concatenate((indices_batch_indices, indices_batch_shuffle), axis=2)

pcl shape  torch.Size([1, 123397, 4])
(123397, 4)
min  [[-64.347  -6.962 -79.833]]
max  [[68.529  2.897 77.739]]
block size  (265.75201416015625, 19.718000411987305, 315.14398193359375)
diff block  [[51.366      6.3919997 75.531    ]
 [31.949001   6.183     99.169    ]
 [76.429      5.858     83.012    ]
 ...
 [71.728      5.248     76.156    ]
 [59.96       5.048     85.912    ]
 [66.715      7.446     87.598    ]]
 blocks  [[0 0 0]
 [0 0 0]
 [0 0 0]
 ...
 [0 0 0]
 [0 0 0]
 [0 0 0]]
(array([[0, 0, 0]]), array([0, 0, 0, ..., 0, 0, 0]), array([123397]))
threshold  50000.0
blocking
[123397]
[0 0 0 ... 0 0 0]
[[0 0 0]]
here
0 123397
(123397,)
(123397, 1)
(123397, 4)
0
(1, 123397, 4)
(20,)
(20, 150000)
1
[[[ 14.874  14.839  -1.605   0.35 ]
  [-20.778  -3.804  -0.991   0.42 ]
  [  5.198  -3.159  -1.687   0.25 ]
  ...
  [  6.882   1.893  -1.737   0.17 ]
  [  4.021  -3.309  -1.707   0.26 ]
  [-31.75    7.92    0.156   0.25 ]]]
(1, 123397, 4)
(1,)
(1, 150000)
(20, 123397, 4)
123397


In [94]:
def data_preprocessing(max_point_num):
    batch_size = 2048
    block_size = 1000
    grid_size = 0.25
        
    
    data = np.zeros((batch_size, max_point_num, 4))
    data_num = np.zeros((batch_size), dtype=np.int32)
    label = np.zeros((batch_size), dtype=np.int32)
    label_seg = np.zeros((batch_size, max_point_num), dtype=np.int32)
    indices_split_to_full = np.zeros((batch_size, max_point_num), dtype=np.int32)


    
    xyzi = np.fromfile("000000.bin",dtype=np.float32).reshape((-1, 4))
    
    
    indices_for_prediction = np.arange(xyzi.shape[0]) #(xyzi[:,0] >= -5 ).nonzero()[0]
    #print("indices_for_prediction", indices_for_prediction)
    # Filter point only in front on of ego-sensors
    xyzif =xyzi #= xyzi[xyzi[:,0] >= -5 ] 
    
    all_label_pred = np.zeros((xyzi.shape[0]),dtype=int)
    label_length = xyzif.shape[0]
    xyz =xyzif[:,0:3]
    
    i =( xyzif[:,3:4] / np.max(xyzif[:,3:4].flatten()) ) 
    
            
            
            
    xyz_min = np.amin(xyz, axis=0, keepdims=True)
    xyz_max = np.amax(xyz, axis=0, keepdims=True)
    block_size = (2 * (xyz_max[0, 0] - xyz_min[0, 0]), 2 * (xyz_max[0, 1] - xyz_min[0, 1]) ,  2 * (xyz_max[0, -1] - xyz_min[0, -1]))
    
    xyz_blocks = np.floor((xyz - xyz_min) / block_size).astype(np.int)

    #print('{}-Collecting points belong to each block...'.format(datetime.now(), xyzrcof.shape[0]))
    blocks, point_block_indices, block_point_counts = np.unique(xyz_blocks, return_inverse=True,
                                                                return_counts=True, axis=0)
    block_point_indices = np.split(np.argsort(point_block_indices), np.cumsum(block_point_counts[:-1]))
    #print('{}-{} is split into {} blocks.'.format(datetime.now(), dataset, blocks.shape[0]))

    block_to_block_idx_map = dict()
    for block_idx in range(blocks.shape[0]):
        block = (blocks[block_idx][0], blocks[block_idx][1])
        block_to_block_idx_map[(block[0], block[1])] = block_idx

    # merge small blocks into one of their big neighbors
    block_point_count_threshold = max_point_num / 3
    #print("block_point_count_threshold",block_point_count_threshold)
    nbr_block_offsets = [(0, 1), (1, 0), (0, -1), (-1, 0), (-1, 1), (1, 1), (1, -1), (-1, -1)]
    block_merge_count = 0
    for block_idx in range(blocks.shape[0]):
        if block_point_counts[block_idx] >= block_point_count_threshold:
            #print(block_idx, block_point_counts[block_idx])

            continue


        block = (blocks[block_idx][0], blocks[block_idx][1])
        for x, y in nbr_block_offsets:
            nbr_block = (block[0] + x, block[1] + y)
            if nbr_block not in block_to_block_idx_map:
                continue

            nbr_block_idx = block_to_block_idx_map[nbr_block]
            if block_point_counts[nbr_block_idx] < block_point_count_threshold:
                continue


            #print(block_idx, nbr_block_idx, block_point_counts[nbr_block_idx])

            block_point_indices[nbr_block_idx] = np.concatenate(
                [block_point_indices[nbr_block_idx], block_point_indices[block_idx]], axis=-1)
            block_point_indices[block_idx] = np.array([], dtype=np.int)
            block_merge_count = block_merge_count + 1
            break
    #print('{}-{} of {} blocks are merged.'.format(datetime.now(), block_merge_count, blocks.shape[0]))

    idx_last_non_empty_block = 0
    for block_idx in reversed(range(blocks.shape[0])):
        if block_point_indices[block_idx].shape[0] != 0:
            idx_last_non_empty_block = block_idx
            break

    # uniformly sample each block
    for block_idx in range(idx_last_non_empty_block + 1):
        point_indices = block_point_indices[block_idx]
        if point_indices.shape[0] == 0:
            continue

        #print(block_idx, point_indices.shape)
        block_points = xyz[point_indices]
        block_min = np.amin(block_points, axis=0, keepdims=True)
        xyz_grids = np.floor((block_points - block_min) / grid_size).astype(np.int)
        grids, point_grid_indices, grid_point_counts = np.unique(xyz_grids, return_inverse=True,
                                                                 return_counts=True, axis=0)
        grid_point_indices = np.split(np.argsort(point_grid_indices), np.cumsum(grid_point_counts[:-1]))
        grid_point_count_avg = int(np.average(grid_point_counts))
        point_indices_repeated = []
        for grid_idx in range(grids.shape[0]):
            point_indices_in_block = grid_point_indices[grid_idx]
            repeat_num = math.ceil(grid_point_count_avg / point_indices_in_block.shape[0])
            if repeat_num > 1:
                point_indices_in_block = np.repeat(point_indices_in_block, repeat_num)
                np.random.shuffle(point_indices_in_block)
                point_indices_in_block = point_indices_in_block[:grid_point_count_avg]
            point_indices_repeated.extend(list(point_indices[point_indices_in_block]))
        block_point_indices[block_idx] = np.array(point_indices_repeated)
        block_point_counts[block_idx] = len(point_indices_repeated)

    idx = 0
    for block_idx in range(idx_last_non_empty_block + 1):
        point_indices = block_point_indices[block_idx]
        if point_indices.shape[0] == 0:
            continue

        block_point_num = point_indices.shape[0]
        block_split_num = int(math.ceil(block_point_num * 1.0 / max_point_num))
        point_num_avg = int(math.ceil(block_point_num * 1.0 / block_split_num))
        point_nums = [point_num_avg] * block_split_num
        point_nums[-1] = block_point_num - (point_num_avg * (block_split_num - 1))
        starts = [0] + list(np.cumsum(point_nums))

        np.random.shuffle(point_indices)
        block_points = xyz[point_indices]


        block_min = np.amin(block_points, axis=0, keepdims=True)
        block_max = np.amax(block_points, axis=0, keepdims=True)
        #block_center = (block_min + block_max) / 2
        #block_center[0][-1] = block_min[0][-1]
        #block_points = block_points - block_center  # align to block bottom center
        x, y, z = np.split(block_points, (1, 2), axis=-1)

        block_xzyrgbi = np.concatenate([x, z, y, i[point_indices]], axis=-1)

        for block_split_idx in range(block_split_num):
            start = starts[block_split_idx]
            point_num = point_nums[block_split_idx]
            #print(block_split_num, block_split_idx, point_num )



            end = start + point_num
            idx_in_batch = idx % batch_size
            data[idx_in_batch, 0:point_num, ...] = block_xzyrgbi[start:end, :]
            data_num[idx_in_batch] = point_num
            indices_split_to_full[idx_in_batch, 0:point_num] = point_indices[start:end]

            #print("indices_split_to_full", idx_in_batch, point_num, indices_split_to_full)

            if  (block_idx == idx_last_non_empty_block and block_split_idx == block_split_num - 1): #Last iteration

                item_num = idx_in_batch + 1
                
            idx = idx + 1
            
    return label_length, data, data_num, indices_split_to_full, item_num, all_label_pred, indices_for_prediction

In [98]:
label_length, data, data_num, indices_split_to_full, item_num, all_label_pred, indices_for_prediction = data_preprocessing(150000)
print(" 1\n" , label_length," 2\n", data, data_num," 3\n", indices_split_to_full,"4 \n", item_num, "5 \n",all_label_pred," 6\n", indices_for_prediction)
print(data.shape)
merged_label_zero = np.zeros((label_length),dtype=int)
merged_confidence_zero = np.zeros((label_length),dtype=float)


data =data[0:item_num, ...].astype(np.float32) 
print("dat ",data.shape)

data_num =data_num[0:item_num, ...] 
print("dat2 ",data_num.shape)
indices_split_to_full = indices_split_to_full[0:item_num]

batch_num = data.shape[0]

labels_pred = np.full((batch_num, max_point_num), -1, dtype=np.int32)
confidences_pred = np.zeros((batch_num, max_point_num), dtype=np.float32)


 1
 123397  2
 [[[-30.64100075   0.52200001   5.66599989   0.14141414]
  [ 34.38199997  -1.91700006  14.93900013   0.        ]
  [ -1.40499997  -1.78100002   4.37599993   0.2020202 ]
  ...
  [  0.           0.           0.           0.        ]
  [  0.           0.           0.           0.        ]
  [  0.           0.           0.           0.        ]]

 [[-11.67199993  -0.62800002  -8.92599964   0.32323232]
  [ -4.28999996  -0.77899998 -12.59899998   0.37373737]
  [-28.0170002   -1.58000004   6.73500013   0.        ]
  ...
  [  0.           0.           0.           0.        ]
  [  0.           0.           0.           0.        ]
  [  0.           0.           0.           0.        ]]

 [[  0.           0.           0.           0.        ]
  [  0.           0.           0.           0.        ]
  [  0.           0.           0.           0.        ]
  ...
  [  0.           0.           0.           0.        ]
  [  0.           0.           0.           0.        ]
  [  0.    

NameError: name 'max_point_num' is not defined